In [396]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import urllib.request, urllib.error, urllib.parse
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import statsmodels.formula.api as sm



In [134]:
#import the listing dataset for Seattle Airbnb

df=pd.read_csv(r"C:\Users\abhis\Downloads\seattle\listings.csv")

In [280]:
#clean the dataset and remove unwanted columns

cleaned_df=df[['id','listing_url','name','host_response_time','host_response_rate',
               'host_is_superhost','host_identity_verified','bed_type','amenities','price','security_deposit',
               'cleaning_fee','cancellation_policy','instant_bookable','extra_people','review_scores_accuracy',
               'review_scores_cleanliness','review_scores_checkin', 'review_scores_communication',
               'review_scores_location', 'review_scores_value','is_location_exact', 'host_acceptance_rate']]

In [21]:
#import the reviews dataset for all Airbnb reviews

reviews_df=pd.read_csv(r"C:\Users\abhis\Downloads\seattle\reviews.csv")


In [22]:
#convering the reviews to string

for i in range (0, len(reviews_df)):
    reviews_df['comments'][i]=str(reviews_df['comments'][i])

C:\Users\abhis\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
#taking reviews which are greater than 3 charaters

len_com=[]
for i in range (0, len(reviews_df)):
    len_com.append(len(reviews_df['comments'][i]))
    
reviews_df['len_com']=len_com

In [25]:
new_df=reviews_df[reviews_df['len_com']>3]

<bound method DataFrame.reset_index of        listing_id        id        date  reviewer_id reviewer_name  \
0         7202016  38917982  2015-07-19     28943674        Bianca   
1         7202016  39087409  2015-07-20     32440555         Frank   
2         7202016  39820030  2015-07-26     37722850           Ian   
3         7202016  40813543  2015-08-02     33671805        George   
4         7202016  41986501  2015-08-10     34959538          Ming   
...           ...       ...         ...          ...           ...   
84844     3624990  50436321  2015-10-12     37419458          Ryan   
84845     3624990  51024875  2015-10-17      6933252         Linda   
84846     3624990  51511988  2015-10-20     19543701         Jaime   
84847     3624990  52814482  2015-11-02     24445024        Jørgen   
84848     9727246  56429621  2015-12-13     30860360     Stephanie   

                                                comments  len_com  
0      Cute and cozy place. Perfect location to ever

In [27]:
#reindexing the dataframe to avoid missing id
new_df.index = range(len(new_df))

In [29]:
#detecting the languages for all the reviews to remove any reviews with unsupported languages

from langdetect import detect
language_comment=[]
for i in range (0, len(new_df)):
    language_comment.append(detect(new_df['comments'][i]))
    
new_df['lang']=language_comment

C:\Users\abhis\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:

lang_df=new_df[new_df['lang']=='en']
lang_df.index = range(len(lang_df))

In [221]:
#generating sentiment score for all the reviews using Google NLP package

sentiment_score=[]

import os 
import io 
from google.cloud import language 
from google.cloud.language import enums 
from google.cloud.language import types 
  
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(r'C:\Users\\abhis\Downloads\\Big Data Technologies-0b73b3f0cc1d.json') 
  
# Instantiates a client
client = language.LanguageServiceClient()

count=0
# The text to analyze
for i in range(0, len(lang_df)):
    if count < 500:
        text = lang_df['comments'][i]
        document = types.Document(
            content=text,
            language='en',
            type=enums.Document.Type.PLAIN_TEXT)
        
        # Detects the sentiment of the text
        sentiment = client.analyze_sentiment(document=document).document_sentiment
        sentiment_score.append(sentiment.score)
        count+=1
    else:
        time.sleep(60)
        count=0
        text = lang_df['comments'][i]
        document = types.Document(
            content=text,
            language='en',
            type=enums.Document.Type.PLAIN_TEXT)

        # Detects the sentiment of the text
        sentiment = client.analyze_sentiment(document=document).document_sentiment
        sentiment_score.append(sentiment.score)
        count+=1

In [226]:
#adding sentiment scores to the reviews dataframe

lang_df['sentiment_score']=sentiment_score

lang_df=lang_df[['listing_id','sentiment_score']]
lang_df=lang_df.groupby(['listing_id']).mean()

C:\Users\abhis\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [299]:
#joining the 2 dataframe. Linking all airbnb with their sentiment scores

cleaned_df=cleaned_df.merge(lang_df, left_on=['id'], right_on=['listing_id'], how='left')

In [282]:
#filling NaN values
cleaned_df=cleaned_df.fillna('00')


In [283]:
#removing , from price
price_list=[]
for i in range (0, len(cleaned_df)):
    price_list.append(cleaned_df['price'][i].replace(',',''))
    
cleaned_df['price']=price_list

In [284]:
#removing $ sign from price

for i in range (0, len(cleaned_df)):
    cleaned_df['price'][i]=float(cleaned_df['price'][i][1:])

C:\Users\abhis\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [285]:
#removing , from security deposit
sec_list=[]
for i in range (0, len(cleaned_df)):
    sec_list.append(cleaned_df['security_deposit'][i].replace(',',''))

cleaned_df['security_deposit']=sec_list


In [286]:
#removing $ sign from security deposit

for i in range (0, len(cleaned_df)):
    cleaned_df['security_deposit'][i]=float(cleaned_df['security_deposit'][i][1:])

C:\Users\abhis\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [287]:
#removing , from cleaning fee
clean_list=[]
for i in range (0, len(cleaned_df)):
    clean_list.append(cleaned_df['cleaning_fee'][i].replace(',',''))
    
cleaned_df['cleaning_fee']=clean_list


In [288]:
#removing $ sign from cleaning_fee


for i in range (0, len(cleaned_df)):
    cleaned_df['cleaning_fee'][i]=float(cleaned_df['cleaning_fee'][i][1:])

C:\Users\abhis\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [289]:
#removing , from extra people
extra_list=[]
for i in range (0, len(cleaned_df)):
    extra_list.append(cleaned_df['extra_people'][i].replace(',',''))

cleaned_df['extra_people']=extra_list


In [290]:
#removing $ sign from extra people

for i in range (0, len(cleaned_df)):
    cleaned_df['extra_people'][i]=float(cleaned_df['extra_people'][i][1:])

C:\Users\abhis\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [295]:
#converting variables from string to float. These were continuous variable.

cleaned_df['price'] = cleaned_df['price'].apply(lambda r: float(r))
cleaned_df['security_deposit'] = cleaned_df['security_deposit'].apply(lambda r: float(r))
cleaned_df['cleaning_fee'] = cleaned_df['cleaning_fee'].apply(lambda r: float(r))
cleaned_df['extra_people'] = cleaned_df['extra_people'].apply(lambda r: float(r))
cleaned_df['review_scores_accuracy'] = cleaned_df['review_scores_accuracy'].apply(lambda r: float(r))
cleaned_df['review_scores_checkin'] = cleaned_df['review_scores_checkin'].apply(lambda r: float(r))
cleaned_df['review_scores_cleanliness'] = cleaned_df['review_scores_cleanliness'].apply(lambda r: float(r))
cleaned_df['review_scores_communication'] = cleaned_df['review_scores_communication'].apply(lambda r: float(r))
cleaned_df['review_scores_location'] = cleaned_df['review_scores_location'].apply(lambda r: float(r))
cleaned_df['review_scores_value'] = cleaned_df['review_scores_value'].apply(lambda r: float(r))
cleaned_df['host_acceptance_rate'] = cleaned_df['host_acceptance_rate'].apply(lambda r: float(r))
cleaned_df['host_response_rate'] = cleaned_df['host_response_rate'].apply(lambda r: float(r))

In [306]:
#exporting the final used dataframe to a csv. Can be used directly to make the equation
cleaned_df.to_csv("D:\Purdue Courses\Fall Mod 2\cleaned_df.csv")